In [ ]:
!pip install -r requirements.txt --user

In [8]:
import boto3
import time

sm = boto3.Session().client('sagemaker')

In [ ]:
def get_regions():
    ec2 = boto3.client("ec2")
    regions = ec2.describe_regions()
    return [r["RegionName"] for r in regions["Regions"]]

In [9]:
def cleanup_experiment(_experiment_name):
    trials = sm.list_trials(ExperimentName=_experiment_name)['TrialSummaries']
    print('TrialNames:')
    for trial in trials:
        trial_name = trial['TrialName']
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print('\tTrialComponentNames:')
        for component in components_in_trial['TrialComponentSummaries']:
            component_name = component['TrialComponentName']
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=_experiment_name)
    print(f"\nExperiment {_experiment_name} deleted")

In [ ]:
def cleanup_monitoring_schedules(_monitoring_schedule_name):
    sm.delete_monitoring_schedule(
        MonitoringScheduleName=_monitoring_schedule_name
    )

In [ ]:
def cleanup_endpoints(_endpoint_name):
    sm.delete_endpoint(EndpointName=_endpoint_name)

In [19]:
experiment_name = "Experiment-1"
cleanup_experiment(experiment_name)

sm.list_monitoring_schedules()
monitoring_schedules = sm.list_monitoring_schedules()
for monitoring_schedule in monitoring_schedules['MonitoringScheduleSummaries']:
    print(monitoring_schedule['MonitoringScheduleName'])
    monitoring_schedule_name = monitoring_schedule['MonitoringScheduleName']
    cleanup_monitoring_schedules(monitoring_schedule_name)

endpoints = sm.list_endpoints()
for endpoint in endpoints['Endpoints']:
    print(endpoint['EndpointName'])
    endpoint_name = endpoint['EndpointName']
    cleanup_endpoints(endpoint_name)

TrialNames:

Experiment Experiment-1 deleted


In [ ]:
get_regions()